In [1]:
import os
import sys
import time
sys.path.extend(['..'])

import torch
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{amssymb}')

In [3]:
res_path = '../../results/pretrained'

ece_file = 'logitsJuan_ECE_reg_{}.csv'
nll_file = 'logitsJuan_NLL_reg_{}.csv'
bri_file = 'logitsJuan_BRI_reg_{}.csv'

In [4]:
ECEs = [pd.read_csv(os.path.join(res_path, ece_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]
NLLs = [pd.read_csv(os.path.join(res_path, nll_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]
Bris = [pd.read_csv(os.path.join(res_path, bri_file.format(i))).drop(['Unnamed: 0'], axis=1).set_index(['Dataset', 'Model']) for i in range(10)]

In [11]:
ECE = sum(ECEs)/len(ECEs)
NLL = sum(NLLs)/len(NLLs)
Bri = sum(Bris)/len(Bris)

In [12]:
def highlight_min(s):
    '''
    highlight the minimum.
    '''
    is_max = s == s.min()
    return ['font-weight: bold' if v else '' for v in is_max]

In [13]:
def highlight_min_br(s):
    print(s['ECE'])
    
    
    s['ECE'].style.apply(highlight_min, subset=s.select_dtypes(float))
    s['NLL'].style.apply(highlight_min, subset=s.select_dtypes(float))
    s['Brier'].style.apply(highlight_min, subset=s.select_dtypes(float))
      
    return s

### CIFAR 10

In [14]:
curr_ece = ECE.loc['cifar10']
curr_nll = NLL.loc['cifar10']
curr_bri = Bri.loc['cifar10']

In [18]:
curr_ece

,Uncalibrated,TS,ETS,MIR,BTS,PTS,PTS_ece,LTS,HTS,HnLTS,LTS_torch,HTS_torch,HnLTS_torch
Model,,,,,,,,,,,,,
densenet-121,2.835130,1.386325,2.257274,1.425367,1.091841,1.184769,3.222835,1.438661,1.288794,1.349832,1.413890,1.376855,1.346057
densenet-169,2.791077,1.650473,2.309780,1.355161,1.266717,0.940874,2.120460,1.291028,1.295456,1.212882,1.097763,1.034405,1.020560
resnet-50,10.709518,2.544719,2.837071,1.180272,1.864384,1.657060,3.707049,2.524410,2.426896,1.454847,1.421409,1.583076,2.056772
resnet-101,4.450539,1.373523,1.491050,1.927313,1.718641,1.416300,4.955468,1.336975,1.420545,1.696917,1.604791,1.598841,1.431198
resnext-29_8x16,2.883827,1.026017,0.852116,1.092094,1.185582,1.031849,3.005487,1.079328,1.119375,0.927186,1.036549,1.183239,1.142596
vgg-19,4.605082,2.514875,3.304092,1.630511,1.375733,1.394660,4.905917,2.102290,2.161324,1.651571,1.618061,1.735882,1.665568
wide-resnet-28x10,1.929727,0.697262,1.593208,1.268281,1.071903,0.887326,1.264354,0.791909,0.887552,0.727414,0.890977,0.841236,0.870313
wide-resnet-40x10,3.115250,1.041778,1.137043,1.228119,1.275605,1.046703,3.345515,1.069909,1.203251,1.114712,1.106251,1.076345,1.205023


In [19]:
curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,2.835130,2.791077,10.709518,4.450539,2.883827,4.605082,1.929727,3.115250
TS,1.386325,1.650473,2.544719,1.373523,1.026017,2.514875,0.697262,1.041778
ETS,2.257274,2.309780,2.837071,1.491050,0.852116,3.304092,1.593208,1.137043
MIR,1.425367,1.355161,1.180272,1.927313,1.092094,1.630511,1.268281,1.228119
BTS,1.091841,1.266717,1.864384,1.718641,1.185582,1.375733,1.071903,1.275605
HTS_torch,1.376855,1.034405,1.583076,1.598841,1.183239,1.735882,0.841236,1.076345
LTS_torch,1.413890,1.097763,1.421409,1.604791,1.036549,1.618061,0.890977,1.106251
HnLTS_torch,1.346057,1.020560,2.056772,1.431198,1.142596,1.665568,0.870313,1.205023
PTS,1.184769,0.940874,1.657060,1.416300,1.031849,1.394660,0.887326,1.046703
PTS_ece,3.222835,2.120460,3.707049,4.955468,3.005487,4.905917,1.264354,3.345515


In [20]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex()) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &        TS &       ETS &       MIR &       BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &       PTS &   PTS\_ece \\
Model             &               &           &           &           &           &            &            &              &           &           \\
\midrule
densenet-121      &      2.835130 &  1.386325 &  2.257274 &  1.425367 &  1.091841 &   1.376855 &   1.413890 &     1.346057 &  1.184769 &  3.222835 \\
densenet-169      &      2.791077 &  1.650473 &  2.309780 &  1.355161 &  1.266717 &   1.034405 &   1.097763 &     1.020560 &  0.940874 &  2.120460 \\
resnet-50         &     10.709518 &  2.544719 &  2.837071 &  1.180272 &  1.864384 &   1.583076 &   1.421409 &     2.056772 &  1.657060 &  3.707049 \\
resnet-101        &      4.450539 &  1.373523 &  1.491050 &  1.927313 &  1.718641 &   1.598841 &   1.604791 &     1.431198 &  1.416300 &  4.955468 \\
resnext-29\_8x16   &      2.883827 &  1.026017 &  0.852116 &  1.

In [21]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.2f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &   TS &  ETS &  MIR &  BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &  PTS &  PTS\_ece \\
Model             &               &      &      &      &      &            &            &              &      &          \\
\midrule
densenet-121      &          2.84 & 1.39 & 2.26 & 1.43 & 1.09 &       1.38 &       1.41 &         1.35 & 1.18 &     3.22 \\
densenet-169      &          2.79 & 1.65 & 2.31 & 1.36 & 1.27 &       1.03 &       1.10 &         1.02 & 0.94 &     2.12 \\
resnet-50         &         10.71 & 2.54 & 2.84 & 1.18 & 1.86 &       1.58 &       1.42 &         2.06 & 1.66 &     3.71 \\
resnet-101        &          4.45 & 1.37 & 1.49 & 1.93 & 1.72 &       1.60 &       1.60 &         1.43 & 1.42 &     4.96 \\
resnext-29\_8x16   &          2.88 & 1.03 & 0.85 & 1.09 & 1.19 &       1.18 &       1.04 &         1.14 & 1.03 &     3.01 \\
vgg-19            &          4.61 & 2.51 & 3.30 & 1.63 & 1.38 &       1.74 &       1.62 &      

In [27]:
curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.188129,0.187019,0.789676,0.304690,0.199952,0.299754,0.149671,0.206792
TS,0.161812,0.160774,0.447289,0.216332,0.162096,0.235521,0.136182,0.163392
ETS,0.167319,0.168556,0.451503,0.219902,0.161691,0.242422,0.142409,0.164236
MIR,0.168084,0.164048,0.443231,0.218557,0.171089,0.225514,0.149453,0.165278
BTS,0.193055,0.236106,0.449789,0.256801,0.204134,0.290246,0.210356,0.183968
HTS_torch,0.160669,0.158660,0.438665,0.215263,0.163187,0.230331,0.135608,0.162632
LTS_torch,0.161063,0.158993,0.444694,0.217398,0.162189,0.233029,0.136363,0.163369
HnLTS_torch,0.160418,0.158388,0.438344,0.214440,0.163422,0.229903,0.135504,0.163300
PTS,0.158506,0.154662,0.444678,0.216767,0.162224,0.229363,0.134498,0.160748
PTS_ece,0.237736,0.179673,0.499821,0.417960,0.258632,0.401320,0.139704,0.268326


In [28]:
print(curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.1881 & 0.1618 & 0.1673 & 0.1681 & 0.1931 &     0.1607 &     0.1611 &       0.1604 & 0.1585 &   0.2377 \\
densenet-169      &        0.1870 & 0.1608 & 0.1686 & 0.1640 & 0.2361 &     0.1587 &     0.1590 &       0.1584 & 0.1547 &   0.1797 \\
resnet-50         &        0.7897 & 0.4473 & 0.4515 & 0.4432 & 0.4498 &     0.4387 &     0.4447 &       0.4383 & 0.4447 &   0.4998 \\
resnet-101        &        0.3047 & 0.2163 & 0.2199 & 0.2186 & 0.2568 &     0.2153 &     0.2174 &       0.2144 & 0.2168 &   0.4180 \\
resnext-29\_8x16   &        0.2000 & 0.1621 & 0.1617 & 0.1711 & 0.2041 &     0.1632 &     0.1622 &       0.1634 & 0.1622 &   0.2586 \\
vgg-19            &      

In [29]:
curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.076359,0.075375,0.239194,0.110169,0.082816,0.110094,0.062887,0.081958
TS,0.072888,0.071723,0.203659,0.101096,0.078307,0.101859,0.060798,0.076777
ETS,0.072440,0.071210,0.202883,0.101027,0.078233,0.100686,0.060941,0.076449
MIR,0.072387,0.071438,0.202246,0.101698,0.078487,0.099821,0.061136,0.076852
BTS,0.072232,0.071064,0.203574,0.101902,0.079086,0.100335,0.061111,0.077036
HTS_torch,0.072440,0.070940,0.201083,0.100291,0.078223,0.099873,0.060974,0.076590
LTS_torch,0.072570,0.071170,0.202629,0.101085,0.078369,0.100528,0.060712,0.076710
HnLTS_torch,0.072449,0.070962,0.201293,0.100342,0.078333,0.100021,0.060561,0.076662
PTS,0.072006,0.070627,0.202578,0.100694,0.078000,0.099801,0.060521,0.076413
PTS_ece,0.078851,0.073378,0.209853,0.114408,0.085620,0.113016,0.061338,0.084488


In [30]:
print(curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.0764 & 0.0729 & 0.0724 & 0.0724 & 0.0722 &     0.0724 &     0.0726 &       0.0724 & 0.0720 &   0.0789 \\
densenet-169      &        0.0754 & 0.0717 & 0.0712 & 0.0714 & 0.0711 &     0.0709 &     0.0712 &       0.0710 & 0.0706 &   0.0734 \\
resnet-50         &        0.2392 & 0.2037 & 0.2029 & 0.2022 & 0.2036 &     0.2011 &     0.2026 &       0.2013 & 0.2026 &   0.2099 \\
resnet-101        &        0.1102 & 0.1011 & 0.1010 & 0.1017 & 0.1019 &     0.1003 &     0.1011 &       0.1003 & 0.1007 &   0.1144 \\
resnext-29\_8x16   &        0.0828 & 0.0783 & 0.0782 & 0.0785 & 0.0791 &     0.0782 &     0.0784 &       0.0783 & 0.0780 &   0.0856 \\
vgg-19            &      

### CIFAR 100

In [31]:
curr_ece = ECE.loc['cifar100']
curr_nll = NLL.loc['cifar100']
curr_bri = Bri.loc['cifar100']

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

Metric                     ECE                                          \
Model             Uncalibrated        TS       ETS       MIR       BTS   
Model                                                                    
densenet-121          8.760250  3.933630  3.039655  4.738307  2.768561   
densenet-169          8.932411  3.948774  2.897083  4.996996  3.187847   
resnet-101           11.446499  2.246798  2.243738  3.895105  2.224123   
resnext-29_8x16       9.692425  3.139508  2.675645  4.209867  2.061077   
vgg-19               17.631318  5.133481  5.364392  4.147693  3.885672   
wide-resnet-28x10     5.187933  4.629842  3.546959  3.919492  3.105283   
wide-resnet-40x10    14.784526  4.201723  2.739724  5.343658  3.550641   

Metric                                                               ...  \
Model                   PTS   PTS_ece       LTS       HTS     HnLTS  ...   
Model                                                                ...   
densenet-121       3.085512  7.664987  4.198000  3.978460  3.313878  ...   
densenet-169       3.041095  3.217014  4.229603  4.147906  3.314126  ...   
resnet-101         2.054936  2.434454  2.375905  2.619516  2.268631  ...   
resnext-29_8x16    2.018209  3.350183  3.251355  3.403475  2.021386  ...   
vgg-19             3.746967  6.881733  3.728082  3.123186  3.621421  ...   
wide-resnet-28x10  3.109856  8.353677  4.603869  4.372267  3.574850  ...   
wide-resnet-40x10  3.957800  5.672038  4.425745  4.369427  3.730715  ...   

Metric                Brier                                                    \
Model                   MIR       BTS       PTS   PTS_ece       LTS       HTS   
Model                                                                           
densenet-121       0.304987  0.305248  0.302624  0.318651  0.303898  0.303609   
densenet-169       0.301862  0.301962  0.298267  0.299389  0.299873  0.299584   
resnet-101         0.384248  0.382470  0.380823  0.381413  0.382163  0.381370   
resnext-29_8x16    0.311620  0.309646  0.307139  0.312158  0.309433  0.309821   
vgg-19             0.388874  0.388022  0.385325  0.398534  0.386526  0.388702   
wide-resnet-28x10  0.285417  0.287132  0.282832  0.299460  0.285669  0.285075   
wide-resnet-40x10  0.330068  0.329314  0.325058  0.336461  0.324544  0.324022   

Metric                                                       
Model                 HnLTS LTS_torch HTS_torch HnLTS_torch  
Model                                                        
densenet-121       0.304911  0.305473  0.303184    0.304013  
densenet-169       0.301612  0.302107  0.299293    0.299875  
resnet-101         0.381698  0.381999  0.382068    0.381145  
resnext-29_8x16    0.309001  0.309244  0.308849    0.310063  
vgg-19             0.389501  0.390504  0.386308    0.388808  
wide-resnet-28x10  0.287185  0.287549  0.284323    0.284065  
wide-resnet-40x10  0.328590  0.329496  0.324243    0.324436  

[7 rows x 39 columns]

In [32]:
curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,8.760250,8.932411,11.446499,9.692425,17.631318,5.187933,14.784526
TS,3.933630,3.948774,2.246798,3.139508,5.133481,4.629842,4.201723
ETS,3.039655,2.897083,2.243738,2.675645,5.364392,3.546959,2.739724
MIR,4.738307,4.996996,3.895105,4.209867,4.147693,3.919492,5.343658
BTS,2.768561,3.187847,2.224123,2.061077,3.885672,3.105283,3.550641
HTS_torch,3.480141,3.746898,2.252193,2.738368,3.650830,3.867962,4.004588
LTS_torch,3.377046,3.565910,2.430060,2.266964,4.265725,3.760424,4.063796
HnLTS_torch,3.420780,3.852649,2.153253,2.690024,3.852471,3.628759,4.011166
PTS,3.085512,3.041095,2.054936,2.018209,3.746967,3.109856,3.957800
PTS_ece,7.664987,3.217014,2.434454,3.350183,6.881733,8.353677,5.672038


In [33]:
print(curr_ece[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.2f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &   TS &  ETS &  MIR &  BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &  PTS &  PTS\_ece \\
Model             &               &      &      &      &      &            &            &              &      &          \\
\midrule
densenet-121      &          8.76 & 3.93 & 3.04 & 4.74 & 2.77 &       3.48 &       3.38 &         3.42 & 3.09 &     7.66 \\
densenet-169      &          8.93 & 3.95 & 2.90 & 5.00 & 3.19 &       3.75 &       3.57 &         3.85 & 3.04 &     3.22 \\
resnet-101        &         11.45 & 2.25 & 2.24 & 3.90 & 2.22 &       2.25 &       2.43 &         2.15 & 2.05 &     2.43 \\
resnext-29\_8x16   &          9.69 & 3.14 & 2.68 & 4.21 & 2.06 &       2.74 &       2.27 &         2.69 & 2.02 &     3.35 \\
vgg-19            &         17.63 & 5.13 & 5.36 & 4.15 & 3.89 &       3.65 &       4.27 &         3.85 & 3.75 &     6.88 \\
wide-resnet-28x10 &          5.19 & 4.63 & 3.55 & 3.92 & 3.11 &       3.87 &       3.76 &      

In [34]:
curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.893868,0.874816,1.134310,0.939844,1.541421,0.817338,1.224773
TS,0.835487,0.815570,1.000677,0.822038,1.199659,0.813459,0.905464
ETS,0.859196,0.832331,1.009053,0.839845,1.206897,0.830835,0.926263
MIR,0.837843,0.816676,1.010458,0.833204,1.186490,0.799622,0.925424
BTS,0.827085,0.808159,1.003960,0.818997,1.192554,0.787617,0.903724
HTS_torch,0.819009,0.801315,1.006663,0.815291,1.175813,0.778474,0.883636
LTS_torch,0.825620,0.806293,1.000965,0.812957,1.194686,0.787287,0.900384
HnLTS_torch,0.820924,0.801946,1.002089,0.818886,1.184016,0.775277,0.884446
PTS,0.819410,0.795119,0.998886,0.807010,1.176882,0.769490,0.886692
PTS_ece,0.963806,0.812588,1.002071,0.856577,1.334389,0.916873,1.049804


In [38]:
print(curr_nll[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.8939 & 0.8355 & 0.8592 & 0.8378 & 0.8271 &     0.8190 &     0.8256 &       0.8209 & 0.8194 &   0.9638 \\
densenet-169      &        0.8748 & 0.8156 & 0.8323 & 0.8167 & 0.8082 &     0.8013 &     0.8063 &       0.8019 & 0.7951 &   0.8126 \\
resnet-101        &        1.1343 & 1.0007 & 1.0091 & 1.0105 & 1.0040 &     1.0067 &     1.0010 &       1.0021 & 0.9989 &   1.0021 \\
resnext-29\_8x16   &        0.9398 & 0.8220 & 0.8398 & 0.8332 & 0.8190 &     0.8153 &     0.8130 &       0.8189 & 0.8070 &   0.8566 \\
vgg-19            &        1.5414 & 1.1997 & 1.2069 & 1.1865 & 1.1926 &     1.1758 &     1.1947 &       1.1840 & 1.1769 &   1.3344 \\
wide-resnet-28x10 &      

In [39]:
curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-101,resnext-29_8x16,vgg-19,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.317058,0.314196,0.405327,0.327484,0.443283,0.289225,0.370023
TS,0.304773,0.301670,0.381723,0.309558,0.391811,0.288609,0.327633
ETS,0.304354,0.301037,0.381624,0.309294,0.391021,0.287947,0.326992
MIR,0.304987,0.301862,0.384248,0.311620,0.388874,0.285417,0.330068
BTS,0.305248,0.301962,0.382470,0.309646,0.388022,0.287132,0.329314
HTS_torch,0.303184,0.299293,0.382068,0.308849,0.386308,0.284323,0.324243
LTS_torch,0.305473,0.302107,0.381999,0.309244,0.390504,0.287549,0.329496
HnLTS_torch,0.304013,0.299875,0.381145,0.310063,0.388808,0.284065,0.324436
PTS,0.302624,0.298267,0.380823,0.307139,0.385325,0.282832,0.325058
PTS_ece,0.318651,0.299389,0.381413,0.312158,0.398534,0.299460,0.336461


In [40]:
print(curr_bri[['Uncalibrated', 'TS', 'ETS', 'MIR', 'BTS', 'HTS_torch', 'LTS_torch', 'HnLTS_torch', 'PTS', 'PTS_ece']].to_latex(float_format="%.4f")) 

\begin{tabular}{lrrrrrrrrrr}
\toprule
{} &  Uncalibrated &     TS &    ETS &    MIR &    BTS &  HTS\_torch &  LTS\_torch &  HnLTS\_torch &    PTS &  PTS\_ece \\
Model             &               &        &        &        &        &            &            &              &        &          \\
\midrule
densenet-121      &        0.3171 & 0.3048 & 0.3044 & 0.3050 & 0.3052 &     0.3032 &     0.3055 &       0.3040 & 0.3026 &   0.3187 \\
densenet-169      &        0.3142 & 0.3017 & 0.3010 & 0.3019 & 0.3020 &     0.2993 &     0.3021 &       0.2999 & 0.2983 &   0.2994 \\
resnet-101        &        0.4053 & 0.3817 & 0.3816 & 0.3842 & 0.3825 &     0.3821 &     0.3820 &       0.3811 & 0.3808 &   0.3814 \\
resnext-29\_8x16   &        0.3275 & 0.3096 & 0.3093 & 0.3116 & 0.3096 &     0.3088 &     0.3092 &       0.3101 & 0.3071 &   0.3122 \\
vgg-19            &        0.4433 & 0.3918 & 0.3910 & 0.3889 & 0.3880 &     0.3863 &     0.3905 &       0.3888 & 0.3853 &   0.3985 \\
wide-resnet-28x10 &      

# OLD

### Cars

In [90]:
curr_ece = ECE.loc[ECE.Dataset=='cars'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='cars'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='cars'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

Metric           ECE                                                       \
Model        Dataset Uncalibrated        TS       ETS       MIR       BTS   
Model                                                                       
densenet-121    cars     5.865746  2.342571  1.795338  2.906047  2.161027   
densenet-169    cars     5.821673  2.380512  1.952333  3.064580  2.568552   
resnet-18       cars     7.038802  1.874210  2.563862  3.184676  2.807826   
resnet-50       cars     5.190265  2.481779  1.506835  2.479405  2.610410   
resnet-101      cars     5.400409  2.312754  1.797904  2.318862  2.133736   

Metric                                                ...     Brier            \
Model              PTS   PTS_ece       LTS       HTS  ...       MIR       BTS   
Model                                                 ...                       
densenet-121  1.722200  2.516041  4.381359  3.024916  ...  0.166519  0.166408   
densenet-169  2.625037  3.413900  4.633774  3.037256  ...  0.165727  0.167407   
resnet-18     2.427751  6.532263  4.661463  3.046411  ...  0.196493  0.199192   
resnet-50     2.994893  2.302823  4.791060  2.588476  ...  0.153900  0.156919   
resnet-101    1.875768  7.291529  3.983332  2.612310  ...  0.154523  0.157109   

Metric                                                                    \
Model              PTS   PTS_ece       LTS       HTS     HnLTS LTS_torch   
Model                                                                      
densenet-121  0.164753  0.165410  0.177144  0.167052  0.164132  0.171022   
densenet-169  0.162831  0.165174  0.178780  0.168812  0.162922  0.170277   
resnet-18     0.196377  0.206226  0.208044  0.197072  0.194221  0.201498   
resnet-50     0.153924  0.153920  0.167468  0.155652  0.153273  0.157945   
resnet-101    0.154183  0.173008  0.166680  0.155528  0.152950  0.158828   

Metric                              
Model        HTS_torch HnLTS_torch  
Model                               
densenet-121  0.176489    0.167648  
densenet-169  0.179120    0.169025  
resnet-18     0.208392    0.197337  
resnet-50     0.169409    0.156156  
resnet-101    0.167641    0.156040  

[5 rows x 42 columns]

In [91]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,5.865746,5.821673,7.038802,5.190265,5.400409
TS,2.342571,2.380512,1.874210,2.481779,2.312754
ETS,1.795338,1.952333,2.563862,1.506835,1.797904
MIR,2.906047,3.064580,3.184676,2.479405,2.318862
BTS,2.161027,2.568552,2.807826,2.610410,2.133736
PTS,1.722200,2.625037,2.427751,2.994893,1.875768
PTS_ece,2.516041,3.413900,6.532263,2.302823,7.291529
LTS_torch,3.675098,4.013015,4.479547,3.125023,3.089634
HTS_torch,4.126354,4.673913,4.770630,5.121110,4.588614
HnLTS_torch,2.704132,3.029084,2.932321,2.647035,2.569912


In [92]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,0.533742,0.534814,0.635785,0.493354,0.493310
TS,0.417259,0.416619,0.492074,0.390955,0.384694
ETS,0.441761,0.434734,0.512542,0.396151,0.395235
MIR,0.425016,0.445568,0.521404,0.421890,0.401355
BTS,0.504977,0.510134,0.664115,0.607957,0.470656
PTS,0.415976,0.412732,0.505110,0.408865,0.384875
PTS_ece,0.418281,0.443467,0.638041,0.388326,0.678180
LTS_torch,0.456298,0.455783,0.541167,0.421261,0.416030
HTS_torch,0.575168,0.850279,0.747025,1.067392,0.645671
HnLTS_torch,0.446970,0.475942,0.536840,0.418875,0.407852


In [93]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,0.173450,0.172543,0.207543,0.160202,0.161740
TS,0.164603,0.163421,0.194467,0.154095,0.153650
ETS,0.164872,0.162912,0.194731,0.153287,0.153035
MIR,0.166519,0.165727,0.196493,0.153900,0.154523
BTS,0.166408,0.167407,0.199192,0.156919,0.157109
PTS,0.164753,0.162831,0.196377,0.153924,0.154183
PTS_ece,0.165410,0.165174,0.206226,0.153920,0.173008
LTS_torch,0.171022,0.170277,0.201498,0.157945,0.158828
HTS_torch,0.176489,0.179120,0.208392,0.169409,0.167641
HnLTS_torch,0.167648,0.169025,0.197337,0.156156,0.156040


### Birds

In [94]:
curr_ece = ECE.loc[ECE.Dataset=='birds'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='birds'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='birds'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

Metric           ECE                                                       \
Model        Dataset Uncalibrated        TS       ETS       MIR       BTS   
Model                                                                       
densenet-121   birds    12.429890  2.940008  3.177977  4.306846  3.561389   
densenet-169   birds    12.649694  3.089719  3.098738  4.596288  4.548998   
resnet-18      birds    15.638467  3.397920  3.834035  6.047435  3.937566   
resnet-50      birds    12.986916  3.416744  3.209112  5.214230  4.291787   
resnet-101     birds    12.642524  2.837463  3.192655  5.385260  3.864756   

Metric                                                 ...     Brier  \
Model              PTS    PTS_ece       LTS       HTS  ...       MIR   
Model                                                  ...             
densenet-121  2.890360  14.388830  7.703473  4.586972  ...  0.327244   
densenet-169  2.559659   6.301814  8.913095  4.387460  ...  0.318990   
resnet-18     3.353479  21.639745  9.101882  5.018642  ...  0.376872   
resnet-50     2.907789  19.694574  7.382448  4.083497  ...  0.325711   
resnet-101    4.764418  14.319949  8.876726  4.184777  ...  0.310454   

Metric                                                                    \
Model              BTS       PTS   PTS_ece       LTS       HTS     HnLTS   
Model                                                                      
densenet-121  0.329046  0.322501  0.365645  0.355864  0.329576  0.323027   
densenet-169  0.324452  0.314553  0.321666  0.350502  0.321184  0.314144   
resnet-18     0.374576  0.370457  0.472650  0.410515  0.377509  0.370251   
resnet-50     0.329486  0.320010  0.422586  0.348203  0.322907  0.321320   
resnet-101    0.311545  0.307426  0.349501  0.339734  0.309445  0.305333   

Metric                                        
Model        LTS_torch HTS_torch HnLTS_torch  
Model                                         
densenet-121  0.330846  0.359670    0.331048  
densenet-169  0.322514  0.358431    0.323444  
resnet-18     0.381369  0.414287    0.378883  
resnet-50     0.331713  0.447188    0.323952  
resnet-101    0.314910  0.342638    0.312077  

[5 rows x 42 columns]

In [95]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,12.429890,12.649694,15.638467,12.986916,12.642524
TS,2.940008,3.089719,3.397920,3.416744,2.837463
ETS,3.177977,3.098738,3.834035,3.209112,3.192655
MIR,4.306846,4.596288,6.047435,5.214230,5.385260
BTS,3.561389,4.548998,3.937566,4.291787,3.864756
PTS,2.890360,2.559659,3.353479,2.907789,4.764418
PTS_ece,14.388830,6.301814,21.639745,19.694574,14.319949
LTS_torch,6.408002,5.917763,7.439505,6.942747,6.569755
HTS_torch,9.106183,11.193807,10.808414,24.153763,9.985612
HnLTS_torch,4.377864,4.902950,5.234135,3.798688,4.132398


In [96]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,1.196887,1.184059,1.450385,1.200745,1.169428
TS,0.900072,0.867165,1.035979,0.878792,0.837691
ETS,0.901693,0.867903,1.040046,0.874402,0.839318
MIR,0.978267,0.899091,1.086031,0.940650,0.870990
BTS,1.159346,1.269112,1.141942,1.209747,1.108864
PTS,0.895374,0.868241,1.033661,0.867702,0.858680
PTS_ece,1.378288,0.939876,inf,inf,1.351493
LTS_torch,0.950042,0.909707,1.089677,0.921009,0.890148
HTS_torch,inf,inf,inf,1.415880,inf
HnLTS_torch,1.010287,0.984211,1.135566,0.918255,0.890713


In [97]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-18,resnet-50,resnet-101
Uncalibrated,0.352798,0.347038,0.417436,0.358351,0.337751
TS,0.323056,0.314235,0.370182,0.321053,0.305229
ETS,0.323283,0.314540,0.371532,0.321442,0.305637
MIR,0.327244,0.318990,0.376872,0.325711,0.310454
BTS,0.329046,0.324452,0.374576,0.329486,0.311545
PTS,0.322501,0.314553,0.370457,0.320010,0.307426
PTS_ece,0.365645,0.321666,0.472650,0.422586,0.349501
LTS_torch,0.330846,0.322514,0.381369,0.331713,0.314910
HTS_torch,0.359670,0.358431,0.414287,0.447188,0.342638
HnLTS_torch,0.331048,0.323444,0.378883,0.323952,0.312077


### SVHN

In [88]:
curr_ece = ECE.loc[ECE.Dataset=='svhn'].set_index('Model')
curr_nll = NLL.loc[ECE.Dataset=='svhn'].set_index('Model')
curr_bri = Bri.loc[ECE.Dataset=='svhn'].set_index('Model')

pd.concat([curr_ece, curr_nll, curr_bri], axis=1, keys=['ECE', 'NLL', 'Brier'], names=['Metric', 'Model'])

Metric                ECE                                             \
Model             Dataset Uncalibrated        TS       ETS       MIR   
Model                                                                  
densenet-121         svhn     2.058800  1.445866  2.573994  0.845361   
densenet-169         svhn     0.491088  1.037124  1.156383  1.016361   
resnet-50            svhn     0.956623  1.106313  1.449459  1.044810   
wide-resnet-28x10    svhn     1.553618  1.083106  1.589714  0.903520   
wide-resnet-40x10    svhn     1.330812  1.276230  2.535067  0.853099   

Metric                                                               ...  \
Model                   BTS       PTS   PTS_ece       LTS       HTS  ...   
Model                                                                ...   
densenet-121       0.724117  1.095324  2.638281  1.677259  1.396225  ...   
densenet-169       0.933092  0.852070  1.198361  0.954254  1.266279  ...   
resnet-50          1.006148  1.011055  2.622655  1.043941  1.202050  ...   
wide-resnet-28x10  1.095303  1.033192  2.287509  1.062386  0.845815  ...   
wide-resnet-40x10  0.880129  0.969191  1.549845  1.257521  1.467862  ...   

Metric                Brier                                                    \
Model                   MIR       BTS       PTS   PTS_ece       LTS       HTS   
Model                                                                           
densenet-121       0.052341  0.052084  0.052191  0.058026  0.052383  0.052558   
densenet-169       0.052320  0.052088  0.051900  0.052059  0.051998  0.052255   
resnet-50          0.055794  0.055683  0.055442  0.061196  0.055540  0.055700   
wide-resnet-28x10  0.052946  0.053085  0.052773  0.057173  0.052627  0.052640   
wide-resnet-40x10  0.048224  0.048211  0.047818  0.048508  0.048138  0.048090   

Metric                                                       
Model                 HnLTS LTS_torch HTS_torch HnLTS_torch  
Model                                                        
densenet-121       0.052127  0.052376  0.052118    0.052371  
densenet-169       0.051816  0.051553  0.051911    0.051669  
resnet-50          0.055496  0.055381  0.055474    0.055433  
wide-resnet-28x10  0.052786  0.052928  0.052653    0.052729  
wide-resnet-40x10  0.048094  0.048317  0.048017    0.048169  

[5 rows x 42 columns]

In [89]:
curr_ece.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,2.058800,0.491088,0.956623,1.553618,1.330812
TS,1.445866,1.037124,1.106313,1.083106,1.276230
ETS,2.573994,1.156383,1.449459,1.589714,2.535067
MIR,0.845361,1.016361,1.044810,0.903520,0.853099
BTS,0.724117,0.933092,1.006148,1.095303,0.880129
PTS,1.095324,0.852070,1.011055,1.033192,0.969191
PTS_ece,2.638281,1.198361,2.622655,2.287509,1.549845
LTS_torch,1.177281,0.798178,0.968126,1.093625,1.237495
HTS_torch,0.979599,0.996584,1.043561,1.039999,1.017069
HnLTS_torch,1.229644,0.790663,1.008837,1.010006,1.074584


In [82]:
curr_nll.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.159902,0.127533,0.140505,0.141678,0.128077
TS,0.141412,0.129260,0.140116,0.134269,0.126885
ETS,0.151157,0.129777,0.142070,0.137570,0.137088
MIR,0.138444,0.130222,0.143252,0.133155,0.124451
BTS,0.168346,0.152188,0.151398,0.145151,0.136512
PTS,0.139548,0.128507,0.138841,0.133768,0.125052
PTS_ece,0.243524,0.138088,0.269564,0.182695,0.127343
LTS_torch,0.140261,0.127443,0.138837,0.133522,0.126197
HTS_torch,0.139961,0.128683,0.139213,0.133058,0.125757
HnLTS_torch,0.140220,0.127803,0.139209,0.132619,0.125048


In [81]:
curr_bri.drop('Dataset', axis=1).drop(columns=['LTS', 'HTS', 'HnLTS']).transpose().style.apply(highlight_min)

Model,densenet-121,densenet-169,resnet-50,wide-resnet-28x10,wide-resnet-40x10
Uncalibrated,0.054979,0.051542,0.055650,0.054144,0.049306
TS,0.052406,0.052033,0.055628,0.052815,0.048192
ETS,0.052608,0.052127,0.055899,0.053107,0.048666
MIR,0.052341,0.052320,0.055794,0.052946,0.048224
BTS,0.052084,0.052088,0.055683,0.053085,0.048211
PTS,0.052191,0.051900,0.055442,0.052773,0.047818
PTS_ece,0.058026,0.052059,0.061196,0.057173,0.048508
LTS_torch,0.052376,0.051553,0.055381,0.052928,0.048317
HTS_torch,0.052118,0.051911,0.055474,0.052653,0.048017
HnLTS_torch,0.052371,0.051669,0.055433,0.052729,0.048169
